In [1]:
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
lines

,src,tar,lic
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Hi.,Salut.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Cours !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Courez !,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
...,...,...,...
179899,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
179900,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
179901,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
179902,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [4]:
del lines['lic']

In [5]:
lines

,src,tar
0,Go.,Va !
1,Hi.,Salut !
2,Hi.,Salut.
3,Run!,Cours !
4,Run!,Courez !
...,...,...
179899,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
179900,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
179901,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
179902,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [6]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:50000] # 5만개만 저장
lines.sample(10)

,src,tar
37897,I've got to get home.,Il me faut aller à la maison.
41453,Can I sit on your lap?,Est-ce que je peux m'asseoir sur tes cuisses ?
9057,Is that my mug?,Est-ce que c'est ma tasse ?
21782,That must be hard.,Cela doit être dur.
36357,I can't tell you yet.,Je ne peux pas encore te le dire.
36367,I caused an accident.,J'ai provoqué un accident.
44313,I'm trying not to cry.,J'essaye de ne pas pleurer.
46445,Tom was in the shower.,Tom était sous la douche.
41768,Do you walk to school?,Marchez-vous jusqu'à l'école ?
2993,That's icky.,C'est gnangnan.


In [7]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
15353,I don't know her.,\t Je ne la connais pas. \n
15869,I want some help.,\t Je veux de l'aide. \n
25795,I'll call the cops.,\t Je vais appeler la police. \n
40026,We have faith in Tom.,\t Nous avons foi en Tom. \n
35112,Did Tom speak French?,\t Tom parlait-il français ? \n
38602,Raise your left hand.,\t Lève la main gauche. \n
14151,You're safe now.,\t Tu es désormais en sécurité. \n
24927,I can do it myself.,\t Je peux le faire moi-même. \n
47435,You may need a shovel.,\t Tu pourrais avoir besoin d'une pelle. \n
3936,I need a bag.,\t Il me faut un sac. \n


In [8]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

77
104


In [9]:
src_vocab=sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'X': 46, 'Y': 47, 'Z': 48, 'a': 49, 'b': 50, 'c': 51, 'd': 52, 'e': 53, 'f': 54, 'g': 55, 'h': 56, 'i': 57, 'j': 58, 'k': 59, 'l': 60, 'm': 61, 'n': 62, 'o': 63, 'p': 64, 'q': 65, 'r': 66, 's': 67, 't': 68, 'u': 69, 'v': 70, 'w': 71, 'x': 72, 'y': 73, 'z': 74, 'é': 75, '€': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 3

In [12]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)

In [13]:
encoder_input[1]

[30, 57, 10]

In [14]:
decoder_input = []
for line in lines.tar: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(tar_to_index[w]) # 글자를 해당되는 정수로 변환
    decoder_input.append(temp_X)

In [15]:
decoder_input[3]

[1, 3, 29, 67, 73, 70, 71, 103, 4, 3, 2]

In [16]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2], [3, 29, 67, 73, 70, 71, 103, 4, 3, 2], [3, 29, 67, 73, 70, 57, 78, 103, 4, 3, 2]]


In [17]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

23
76


In [19]:
for line in lines.tar:
    print(line)

	 Va ! 

	 Salut ! 

	 Salut. 

	 Cours ! 

	 Courez ! 

	 Qui ? 

	 Ça alors ! 

	 Au feu ! 

	 À l'aide ! 

	 Saute. 

	 Ça suffit ! 

	 Stop ! 

	 Arrête-toi ! 

	 Attends ! 

	 Attendez ! 

	 Poursuis. 

	 Continuez. 

	 Poursuivez. 

	 Bonjour ! 

	 Salut ! 

	 Je comprends. 

	 J'essaye. 

	 J'ai gagné ! 

	 Je l'ai emporté ! 

	 J’ai gagné. 

	 Oh non ! 

	 Attaque ! 

	 Attaquez ! 

	 Santé ! 

	 À votre santé ! 

	 Merci ! 

	 Tchin-tchin ! 

	 Lève-toi. 

	 Lève-toi ! 

	 Va, maintenant. 

	 Allez-y maintenant. 

	 Vas-y maintenant. 

	 J'ai pigé ! 

	 Compris ! 

	 Pigé ? 

	 Compris ? 

	 T'as capté ? 

	 Monte. 

	 Montez. 

	 Serre-moi dans tes bras ! 

	 Serrez-moi dans vos bras ! 

	 Je suis tombée. 

	 Je suis tombé. 

	 J'ai fui. 

	 Je sais. 

	 Je suis parti. 

	 Je suis partie. 

	 J'ai menti. 

	 J'ai perdu. 

	 J’ai payé. 

	 J'ai 19 ans. 

	 Je vais bien. 

	 Ça va. 

	 Écoutez ! 

	 C'est pas possible ! 

	 Impossible ! 

	 Hors de question ! 

	 En aucun cas. 


	 On m'a mis une amende. 

	 On m'a collé une amende. 

	 J'ai pris une prune. 

	 J'ai été verbalisé. 

	 J'ai été verbalisée. 

	 On m'a mis une prune. 

	 Je me suis fait verbaliser. 

	 J'ai pris une contravention. 

	 On m'a mis une contravention. 

	 J'ai pris une contredanse. 

	 On m'a mis une contredanse. 

	 J'ai eu de la chance. 

	 J'ai eu du bol. 

	 J'ai été contrariée. 

	 Je me suis fâchée. 

	 Je me suis fâché. 

	 J'ai été contrarié. 

	 J'imagine que non. 

	 Je suppose que non. 

	 Je devais y aller. 

	 Je déteste les chiens. 

	 Je hais les chiens. 

	 Je déteste le poisson. 

	 Je déteste le golf. 

	 Je déteste le lait. 

	 Je déteste le sable. 

	 Je hais le sable. 

	 Je déteste ça. 

	 Je les déteste. 

	 Je déteste ça. 

	 Je déteste le travail. 

	 Je détestais Tom. 

	 J'ai du liquide. 

	 Je dispose de liquide. 

	 J'ai des yeux. 

	 J'ai de la nourriture. 

	 Je dispose de nourriture. 

	 J'ai des nouvelles. 

	 J'ai du temps. 

	 Je dispose de temps. 


	 Utilise-les tous ! 

	 Utilise-les toutes ! 

	 Utilisez-les tous ! 

	 Utilisez-les toutes ! 

	 Attendez un moment. 

	 Attends un moment ! 

	 Attends Tom. 

	 Attendez Tom. 

	 Attends dehors. 

	 Attendez dehors. 

	 Réveille-toi, Tom ! 

	 Marche avec nous. 

	 Marchez avec nous. 

	 Veux-tu un verre ? 

	 Était-ce drôle ? 

	 Nous avons tous pleuré. 

	 Nous avons toutes pleuré. 

	 Nous présentons nos excuses. 

	 Nous sommes arabes. 

	 Nous sommes heureux. 

	 Nous ne pouvons par l'emporter. 

	 Nous pourrions mourir. 

	 Nous nous sommes sentis bien. 

	 Nous nous entendons bien. 

	 On s'entend bien. 

	 Nous avons eu de la chance. 

	 On a eu de la chance. 

	 Nous nous sommes apprêtés. 

	 Nous nous sommes apprêtées. 

	 Nous avons de l'espoir. 

	 Nous avons de la place. 

	 Nous avons de l'espace. 

	 Nous avons du vin. 

	 Nous savions cela. 

	 Nous vivons ici. 

	 Nous les aimons. 

	 Nous t'aimions. 

	 Nous vous aimions. 

	 Nous nous sommes rencontrés aujourd'hu


	 Nous ne sommes pas satisfaites. 

	 Nous sommes malchanceux. 

	 Nous sommes malchanceuses. 

	 Nous sommes inutiles. 

	 Nous sommes en train d'attendre. 

	 Nous sommes riches. 

	 Nous sommes des gagneurs. 

	 Nous sommes en train de gagner. 

	 Nous sommes en train de travailler. 

	 Nous nous faisons du souci. 

	 Nous sommes arrivés. 

	 Nous n'avons pas le temps. 

	 Nous avons refusé. 

	 Nous l'avons vu. 

	 Étais-tu occupée ? 

	 Étais-tu occupé ? 

	 Étiez-vous occupés ? 

	 Étiez-vous occupées ? 

	 As-tu été blessé ? 

	 As-tu été blessée ? 

	 Avez-vous été blessé ? 

	 Avez-vous été blessée ? 

	 Avez-vous été blessés ? 

	 Avez-vous été blessées ? 

	 T'a-t-on tiré dessus ? 

	 Vous a-t-on tiré dessus ? 

	 Quelle poisse ! 

	 Quel fiasco ! 

	 Quelle histoire ! 

	 Quel soulagement ! 

	 Quel cri ! 

	 Qu'en est-il de nous ? 

	 Quel idiot. 

	 Qu'ai-je fait ? 

	 Qu'est-ce que j'ai fait ? 

	 Qu'est-ce que j'y gagne ? 

	 Combien dois-je ? 

	 Combien est-ce que je

	 Vous écrivez bien. 

	 Tu aimerais Tom. 

	 Vous aimeriez Tom. 

	 Vous allez tous mourir. 

	 Ça se passera bien pour toi. 

	 Ça se passera bien pour vous. 

	 Tu seras en sécurité. 

	 Vous êtes une enfant. 

	 Vous êtes un enfant. 

	 Tu es une enfant. 

	 Tu es un enfant. 

	 Tu es un puritain. 

	 Vous êtes un puritain. 

	 Tu es une puritaine. 

	 Vous êtes une puritaine. 

	 Tu es un voleur. 

	 Tu es une voleuse. 

	 Vous êtes un voleur. 

	 Vous êtes une voleuse. 

	 Vous êtes tous fous. 

	 Vous êtes toutes folles. 

	 Vous êtes tous prêts. 

	 Vous êtes toutes prêtes. 

	 Tu es incroyable. 

	 Vous êtes incroyable. 

	 Vous êtes incroyables. 

	 Tu me fais marrer. 

	 Vous me faites marrer. 

	 Tu es génial. 

	 Vous êtes génial. 

	 Tu es géniale. 

	 Vous êtes géniale. 

	 Tu es insensible. 

	 Vous êtes insensible. 

	 Tu es prudent. 

	 Tu es prudente. 

	 Tu as raison. 

	 Vous êtes insaisissable. 

	 Tu es insaisissable. 

	 Tu es tatillon. 

	 Tu es tatillonne. 

	

...


	 J'ai consulté un docteur. 

	 J'ai convaincu tout le monde. 

	 Je pouvais à peine parler. 

	 Je pouvais à peine tenir debout. 

	 Je pourrais vous tuer, maintenant. 

	 Je pourrais te tuer, maintenant. 

	 J'aurais pu faire cela. 

	 Ça m'est tout à fait égal. 

	 Je m'en soucie comme d'une guigne. 

	 Je n'ai pas pu le terminer. 

	 Je ne pouvais pas le finir. 

	 Je compte sur votre aide. 

	 Je compte sur ton aide. 

	 Je traversai la rue. 

	 Je me suis coupée en me rasant. 

	 J'ai rêvassé toute la journée. 

	 J'ai décidé de ne pas manger. 

	 J'exige de la ponctualité. 

	 J'ai trouvé quelque chose. 

	 Je l'ai fait quelques fois. 

	 Je l'ai fait sans toi. 

	 Je l'ai fait sans vous. 

	 J'ai fait ça tout seul. 

	 J'ai fait ça toute seule. 

	 J'ai fait ça tout seul. 

	 J'ai fait ça à l'école. 

	 J'ai fait ça la semaine dernière. 

	 J'ai fait ça lundi. 

	 Je l'ai fait hier. 

	 J'ai fait ce que j'avais à faire. 

	 J'ai fait ce qui convenait. 

	 J'ai fait ce que tu a

	 Ils étaient tous effrayés. 

	 Elles étaient toutes effrayées. 

	 Ils étaient tous deux saouls. 

	 Elles étaient toutes deux saoules. 

	 Ils étaient saouls tous les deux. 

	 Elles étaient saoules toutes les deux. 

	 Ils étaient frustrés. 

	 Elles étaient frustrées. 

	 Ça ne les a pas fait rire. 

	 Ils furent victorieux. 

	 Elles furent victorieuses. 

	 Ils ont été victorieux. 

	 Elles ont été victorieuses. 

	 Elles n'ont pas été invitées. 

	 Ils n'ont pas été invités. 

	 Ils n'attendront pas longtemps. 

	 Ça ira très bien pour eux. 

	 Ça ira très bien pour elles. 

	 Ils vont bientôt revenir. 

	 Ils vont s'éclater. 

	 Elles vont s'éclater. 

	 Ils devront attendre. 

	 Elles devront attendre. 

	 Ils m'emmèneront. 

	 Elles m'emmèneront. 

	 Ils peuvent chanter. 

	 Ils sont capables de chanter. 

	 Elles ont peur de moi. 

	 Ils ont peur de moi. 

	 Elles ont peur de nous. 

	 Ils sont tous là-dedans. 

	 Elles sont toutes là-dedans. 

	 Ils sont tous les mêmes. 




	 Il habite dans un village. 

	 Il vit en marge de la société. 

	 Il a cherché la clé. 

	 Il a l'air pâlot. 

	 Il ressemble à un cheval. 

	 Il a l'air très inquiet. 

	 Il a l'air fort inquiet. 

	 Il a perdu sa nouvelle montre. 

	 Il adore voyager. 

	 Il adore entreprendre des voyages. 

	 Il m'a confectionné un nouveau costume. 

	 Il a inventé l'histoire. 

	 Il inventa cette histoire. 

	 Il est peut-être souffrant. 

	 Il est peut-être alité. 

	 Il va peut-être rater son train. 

	 Il n'est peut-être pas heureux. 

	 Il pourrait ne pas être content. 

	 Il s'approcha d'elle. 

	 Il s'est approché d'elle. 

	 C'est sûrement un bon garçon. 

	 Il doit avoir passé les cinquante ans. 

	 Il doit avoir la cinquantaine passée. 

	 Il doit avoir plus de 60 ans. 

	 Il doit être très heureux. 

	 Il doit avoir été malade. 

	 Il ne me parle jamais. 

	 Il ne buvait jamais de vin. 

	 Il joue souvent de la guitare. 

	 Il possède beaucoup de terre. 

	 Il a ramassé le livre. 

	 I

	 Elle est fâchée, là. 

	 Dois-je parler le français ? 

	 Dois-je étudier le français ? 

	 Devrais-je prendre le bus ? 

	 Devrions-nous prendre une table ? 

	 Ne devrais-tu pas te rendre chez toi ? 

	 Ne devrais-tu pas aller chez toi ? 

	 Ne devriez-vous pas vous rendre chez vous ? 

	 Ne devriez-vous pas aller chez vous ? 

	 Montre-moi une autre montre. 

	 Montrez-moi une autre montre. 

	 Faites-moi voir une autre montre. 

	 Fais-moi voir une autre montre. 

	 Montre-moi quelque chose de nouveau. 

	 Montrez-moi quelque chose de nouveau. 

	 Montre-moi ton passeport ! 

	 Montrez-moi votre passeport ! 

	 Fermez la porte, s'il vous plaît. 

	 Veuillez fermer la porte. 

	 Ferme la porte, je te prie. 

	 Ferme la porte, s'il te plait. 

	 Je vous prie de fermer la porte. 

	 Veuillez fermer la porte ! 

	 Fermez la porte, je vous prie ! 

	 Ferme la porte, je te prie ! 

	 Je te prie de fermer la porte ! 

	 La simplicité est la clé. 

	 Asseyez-vous sur le banc. 

	 Assieds


	 Ne me sous-estime pas. 

	 Ne me sous-estimez pas. 

	 Ne nous sous-estime pas. 

	 Ne gaspillez pas de munitions. 

	 Ne gaspille pas les munitions. 

	 Ne jette pas ton argent par les fenêtres. 

	 Ne t'en fais pas à propos de ça. 

	 Ne t'inquiète pas pour ça. 

	 Ne t'en fais pas ! C'est facile. 

	 Ne vous en faites pas ! C'est facile. 

	 Ne pas écrire à l'encre rouge. 

	 N'écrivez pas à l'encre rouge. 

	 N'as-tu pas d'amis ? 

	 N'avez-vous pas d'amis ? 

	 Vous ne parlez pas français ? 

	 Ne veux-tu pas donner de l'aide? 

	 Ne voulez-vous pas prêter assistance ? 

	 Ne veux-tu pas savoir ? 

	 Ne voulez-vous pas savoir ? 

	 Rêver ne coûte rien. 

	 Le bois sec brûle vite. 

	 Pâques est à portée de main. 

	 Les Elfes ont les oreilles en pointes. 

	 Profites-en tant que tu peux ! 

	 Profitez-en tant que vous pouvez ! 

	 L'Europe n'est pas un pays. 

	 Même Tom était surpris. 

	 Même un enfant peut le faire. 

	 Chaque personne est unique. 

	 Tout le monde est uniqu

In [20]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [21]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [24]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [25]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [26]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
625/625 [==============================] - 202s 324ms/step - loss: 0.7613 - val_loss: 0.6831
Epoch 2/50
625/625 [==============================] - 189s 303ms/step - loss: 0.4824 - val_loss: 0.5549
Epoch 3/50
625/625 [==============================] - 188s 300ms/step - loss: 0.4079 - val_loss: 0.4934
Epoch 4/50
625/625 [==============================] - 191s 305ms/step - loss: 0.3632 - val_loss: 0.4510
Epoch 5/50
625/625 [==============================] - 196s 314ms/step - loss: 0.3322 - val_loss: 0.4235
Epoch 6/50
625/625 [==============================] - 194s 311ms/step - loss: 0.3090 - val_loss: 0.4053
Epoch 7/50
625/625 [==============================] - 195s 311ms/step - loss: 0.2907 - val_loss: 0.3909
Epoch 8/50
625/625 [==============================] - 185s 295ms/step - loss: 0.2758 - val_loss: 0.3798
Epoch 9/50
625/625 [==============================] - 208s 334ms/step - loss: 0.2636 - val_loss: 0.3745
Epoch 10/50
625/625 [==============================] - 193s 309m

In [27]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [28]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [29]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장:  Bies ! 
-----------------------------------
입력 문장: I left.
정답 문장:  Je suis parti. 
번역기가 번역한 문장:  Je suis parti. 
-----------------------------------
입력 문장: Burn it.
정답 문장:  Brûlez-la. 
번역기가 번역한 문장:  Brûle-le. 
-----------------------------------
입력 문장: Drive on.
정답 문장:  Continue à rouler ! 
번역기가 번역한 문장:  Continuez à rouler ! 
-----------------------------------
입력 문장: Step back.
정답 문장:  Recule ! 
번역기가 번역한 문장:  Rembrez. 
